# Validator Behavior Debugging

This notebook helps you understand and debug exactly what the validators are checking.

## Current Issue

The validators should check files **for the specific participant/session** being processed:
- ✓ Correct: `/data/bids/sub-001/ses-01/dwi/*_dwi.nii.gz`
- ✗ Wrong: `/data/bids/dwi/*_dwi.nii.gz` (searches all participants)

Let's verify this is working correctly.

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Optional

from voxelops.validation.context import ValidationContext
from voxelops.validation.validators import (
    HeudiConvValidator,
    QSIParcValidator,
    QSIPrepValidator,
    QSIReconValidator,
)

## Setup Mock Data Structure

In [ ]:
# Create mock inputs
@dataclass
class MockInputs:
    dicom_dir: Optional[Path] = None
    bids_dir: Optional[Path] = None
    qsiprep_dir: Optional[Path] = None
    qsirecon_dir: Optional[Path] = None
    heuristic: Optional[Path] = None
    output_dir: Optional[Path] = None

# Example paths (adjust for your data)
bids_dir = Path("/data/bids")
participant = "001"
session = "01"  # Set to None if no session

## Test QSIPrep Validator

Let's see exactly where it's looking for files:

In [ ]:
# Create validator and context
validator = QSIPrepValidator()

inputs = MockInputs(bids_dir=bids_dir)

context = ValidationContext(
    procedure_name="qsiprep",
    participant=participant,
    session=session,
    inputs=inputs,
)

# Show context properties
print("Context Information:")
print(f"  Participant label: {context.participant_label}")
print(f"  Session label: {context.session_label}")
print(f"  Input dir: {context.input_dir}")
print(f"  Participant dir: {context.participant_dir}")
print()

### Inspect Each Rule

In [ ]:
print("QSIPrep Pre-Validation Rules:")
print("="*60)

for i, rule in enumerate(validator.pre_rules, 1):
    print(f"\n{i}. {rule.__class__.__name__}")
    print(f"   Name: {rule.name}")
    print(f"   Description: {rule.description}")
    print(f"   Severity: {rule.severity}")

    # Show what the rule is checking
    if hasattr(rule, 'base_dir_attr'):
        print(f"   Base dir attr: {rule.base_dir_attr}")
        print(f"   Pattern: {rule.pattern}")
        print(f"   Min count: {rule.min_count}")

        # Show where it will actually search
        base_dir = rule._get_base_dir(context)
        if base_dir:
            search_dir = base_dir / context.participant_label
            if context.session:
                search_dir = search_dir / context.session_label

            print(f"   → Will search in: {search_dir}")
            print(f"   → Full pattern: {search_dir / rule.pattern}")
            print(f"   → Directory exists: {search_dir.exists()}")

            if search_dir.exists():
                found = list(search_dir.glob(rule.pattern))
                print(f"   → Files found: {len(found)}")
                if found:
                    print(f"      {[f.name for f in found[:3]]}")

### Run Validation and See Results

In [ ]:
# Run pre-validation
report = validator.validate_pre(context)

print("\nValidation Report:")
print("="*60)
print(f"Passed: {report.passed}")
print(f"Total checks: {len(report.results)}")
print(f"Errors: {len(report.errors)}")
print(f"Warnings: {len(report.warnings)}")
print()

# Show each result
for result in report.results:
    status = "✓" if result.passed else "✗"
    print(f"{status} {result.rule_name}")
    print(f"  {result.message}")

    # Show search details for glob rules
    if 'search_dir' in result.details:
        print(f"  Searched in: {result.details['search_dir']}")
        print(f"  Pattern: {result.details.get('pattern', 'N/A')}")
        print(f"  Found: {result.details.get('found_count', 0)} files")
    print()

## Test All Validators

Let's check what each validator is looking for:

In [ ]:
def analyze_validator(validator, procedure_name, test_inputs):
    """Analyze what a validator checks."""
    print(f"\n{'='*60}")
    print(f"{procedure_name.upper()} VALIDATOR")
    print(f"{'='*60}")

    context = ValidationContext(
        procedure_name=procedure_name,
        participant=participant,
        session=session,
        inputs=test_inputs,
    )

    print(f"\nPre-validation rules ({len(validator.pre_rules)}):")
    for i, rule in enumerate(validator.pre_rules, 1):
        print(f"  {i}. {rule.description}")

        # Show glob patterns
        if hasattr(rule, 'pattern'):
            base_dir = getattr(rule, '_get_base_dir', lambda c: None)(context)
            if base_dir:
                search_path = base_dir / context.participant_label
                if context.session:
                    search_path = search_path / context.session_label
                print(f"     → {search_path / rule.pattern}")

    print(f"\nPost-validation rules ({len(validator.post_rules)}):")
    for i, rule in enumerate(validator.post_rules, 1):
        print(f"  {i}. {rule.description}")

# Test each validator
analyze_validator(
    HeudiConvValidator(),
    "heudiconv",
    MockInputs(dicom_dir=Path("/data/dicoms"), heuristic=Path("/data/heuristic.py"))
)

analyze_validator(
    QSIPrepValidator(),
    "qsiprep",
    MockInputs(bids_dir=bids_dir)
)

analyze_validator(
    QSIReconValidator(),
    "qsirecon",
    MockInputs(qsiprep_dir=Path("/data/derivatives/qsiprep"))
)

analyze_validator(
    QSIParcValidator(),
    "qsiparc",
    MockInputs(qsirecon_dir=Path("/data/derivatives/qsirecon"))
)

## Verify Correct Behavior

The validation should:
1. **Look in participant directory**: `/data/bids/sub-001/ses-01/dwi/`
2. **Not look in root**: `/data/bids/dwi/` ← This would find files from any participant

### Expected Search Paths

**QSIPrep** (with session):
```
/data/bids/sub-001/ses-01/dwi/*_dwi.nii.gz
/data/bids/sub-001/ses-01/dwi/*_dwi.bval
/data/bids/sub-001/ses-01/dwi/*_dwi.bvec
/data/bids/sub-001/ses-01/anat/*_T1w.nii.gz
```

**QSIPrep** (without session):
```
/data/bids/sub-001/dwi/*_dwi.nii.gz
/data/bids/sub-001/dwi/*_dwi.bval
/data/bids/sub-001/dwi/*_dwi.bvec
/data/bids/sub-001/anat/*_T1w.nii.gz
```

## Manual Test with Your Data

Replace these paths with your actual data and run:

In [ ]:
# YOUR DATA HERE
my_bids_dir = Path("/path/to/your/bids")
my_participant = "001"
my_session = None  # or "01"

# Test with your data
validator = QSIPrepValidator()
inputs = MockInputs(bids_dir=my_bids_dir)
context = ValidationContext(
    procedure_name="qsiprep",
    participant=my_participant,
    session=my_session,
    inputs=inputs,
)

report = validator.validate_pre(context)

print("Validation Results for Your Data:")
print("="*60)
for result in report.results:
    status = "✓" if result.passed else "✗"
    print(f"{status} {result.rule_description}")
    print(f"  {result.message}")
    if 'search_dir' in result.details:
        print(f"  Searched: {result.details['search_dir']}")
    print()

## Issues to Look For

1. **Wrong search directory**: If it's searching `/data/bids/dwi/` instead of `/data/bids/sub-001/dwi/`
2. **Fallback to base_dir**: If participant dir doesn't exist, it falls back to searching the whole BIDS dir
3. **Pattern issues**: Pattern might not match your file naming

## Next Steps

Based on what you find:
1. Note which validators/rules need fixing
2. Share the output from the cells above
3. We'll fix the validators to search in the correct location